# NIR + T

In [1]:
import tensorflow as tf

2024-03-05 10:33:50.406253: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model_path = "gs://etldata-prod-search-ranking-data-hkwv8r/data/shared/neural_ir/transformers_hqi_loose/batch_inf_output_8gpu/2024-03-03/checkpoints/saved_model_04"

In [3]:
nirt_model = tf.saved_model.load(model_path)

RuntimeError: Op type not registered 'SentencepieceOp' in binary running on 5943. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.

# metrank_esci_ce

In [1]:
# import tensorflow as tf
# import onnx
# from onnx_tf.backend import prepare
# from onnx import version_converter

In [2]:
# # download from https://huggingface.co/metarank/ce-esci-MiniLM-L12-v2/blob/main/pytorch_model.onnx
# onnx_model = onnx.load("model_files/metrank_esci_ce_pytorch_model.onnx")
# # converted_model = version_converter.convert_version(onnx_model, 11)
# tf_rep = prepare(onnx_model)
# tf_rep.export_graph("model_files/metrank_esci_ce.pb")

In [3]:
# from onnx.defs import onnx_opset_version
# onnx_opset_version()

In [4]:
# import torch
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('metarank/esci-MiniLM-L6-v2')

In [5]:
# max_seq_length = 128
# inputs = {"input_ids": torch.ones(1, max_seq_length, dtype=torch.int64),
#         "attention_mask": torch.ones(1, max_seq_length, dtype=torch.int64),
#         "token_type_ids": torch.ones(1, max_seq_length, dtype=torch.int64)}

# symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}

# torch.onnx.export(model, args=tuple(inputs.values()), f='model_files/metrank_esci_ce_pytorch_model.onnx', export_params=True, 
#     input_names=['input_ids', 'attention_mask', 'token_type_ids'], output_names=['last_hidden_state'],
#     dynamic_axes={'input_ids': symbolic_names, 'attention_mask': symbolic_names, 'token_type_ids': symbolic_names})

In [18]:
import tensorflow as tf
from tf_sentence_transformers import SentenceTransformer

tf_model = SentenceTransformer.from_pretrained(
    "metarank/esci-MiniLM-L6-v2", from_pt=True
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [19]:
!cat /Users/yzhang/development/yzhang-adhoc-analysis/venv/lib/python3.10/site-packages/tf_sentence_transformers/__init__.py

import tensorflow as tf
from tensorflow import keras
from transformers import AutoTokenizer, TFAutoModel


class SentenceTransformer(tf.keras.layers.Layer):
    """
    Example usage:

    ```
    layer = SentenceTransformer.from_pretrained(
        "sentence-transformers/all-MiniLM-L6-v2", from_pt=False
    )
    inputs = [
        ["This is a test sentence."],
        ["This is another test sentence."],
    ]
    embedding = layer(inputs)
    # assert embedding.shape == (2, 384)
    ```
    """

    def __init__(self, tokenizer, model, embedding_dim: int, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tokenizer = tokenizer
        self.model = model
        self.embedding_dim = embedding_dim

    @classmethod
    def from_pretrained(
        cls,
        model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
        from_pt=False,
        jit_compile=False,
    ):
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        pretrained_setr_tf =

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
sentences = ["This is an example sentence", "Each sentence is converted"]
tokenized = tf_model.tokenizer(sentences, padding=True, truncation=True, return_tensors="tf")
model_output = tf_model.model(tokenized)[0]
tf_embedding = tf_model.mean_pooling([model_output], tokenized["attention_mask"])

In [24]:
tf_embedding = tf_model(tf.constant([[s] for s in sentences]))

In [26]:
tf.constant([[s] for s in sentences])

<tf.Tensor: shape=(2, 1), dtype=string, numpy=
array([[b'This is an example sentence'],
       [b'Each sentence is converted']], dtype=object)>

In [25]:
tf_embedding

<tf.Tensor: shape=(2, 384), dtype=float32, numpy=
array([[ 4.75225374e-02,  7.28033036e-02,  1.13066537e-02,
         5.18702418e-02,  2.05229763e-02,  1.92286372e-02,
         8.24054852e-02,  2.75489278e-02,  4.23876084e-02,
         5.64849228e-02,  7.69773200e-02, -3.91280428e-02,
         3.59982140e-02, -3.35872290e-03,  6.39602989e-02,
        -3.39689851e-03,  7.46712759e-02, -2.88345050e-02,
        -1.07024170e-01, -9.27212555e-03,  1.04597788e-02,
         4.46256623e-02,  9.00389627e-03,  3.30397338e-02,
        -5.14272116e-02, -9.39246174e-03, -2.85237916e-02,
         6.96453080e-02,  1.18409634e-01, -8.12534913e-02,
        -5.36037087e-02, -1.46959815e-02,  6.57981262e-02,
         1.44396964e-02,  1.94283798e-02,  3.49496156e-02,
        -2.83961147e-02,  7.70205259e-02, -4.03326489e-02,
         1.88497361e-02,  1.83968488e-02, -9.29129645e-02,
        -4.21198308e-02, -2.35152431e-02,  3.36448196e-03,
        -6.95117041e-02, -1.55592235e-02,  2.23276112e-03,
      

In [15]:
from sentence_transformers import SentenceTransformer
torch_model = SentenceTransformer('metarank/esci-MiniLM-L6-v2')

In [16]:
sentences = ["This is an example sentence", "Each sentence is converted"]
torch_embeddings = torch_model.encode(sentences)

In [17]:
torch_embeddings

array([[ 4.75225635e-02,  7.28031695e-02,  1.13066006e-02,
         5.18701822e-02,  2.05229912e-02,  1.92286000e-02,
         8.24053213e-02,  2.75489856e-02,  4.23876047e-02,
         5.64849451e-02,  7.69773051e-02, -3.91280018e-02,
         3.59982327e-02, -3.35869798e-03,  6.39603212e-02,
        -3.39687779e-03,  7.46712834e-02, -2.88346000e-02,
        -1.07024215e-01, -9.27208550e-03,  1.04597472e-02,
         4.46256772e-02,  9.00398660e-03,  3.30397598e-02,
        -5.14272340e-02, -9.39259212e-03, -2.85238158e-02,
         6.96453527e-02,  1.18409678e-01, -8.12534615e-02,
        -5.36037758e-02, -1.46959415e-02,  6.57981858e-02,
         1.44397160e-02,  1.94284171e-02,  3.49496603e-02,
        -2.83961240e-02,  7.70205557e-02, -4.03326750e-02,
         1.88497379e-02,  1.83968637e-02, -9.29128528e-02,
        -4.21197973e-02, -2.35153027e-02,  3.36448941e-03,
        -6.95117041e-02, -1.55592058e-02,  2.23272224e-03,
         2.31524594e-02, -5.00241779e-02, -1.12477534e-0